In [48]:
# type: ignore

import requests
import numpy as np
from PIL import Image

import pandas as pd 
import geopandas as gpd
# import osmnx as ox
# import networkx as nx

import pyproj
from shapely.ops import transform

import shapely
from shapely import Point

# import boto3
import os
import django

pd.set_option('display.precision', 2)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "water.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from water.models import ReservoirState, Reservoir, ReservoirStateSerializer, RainFall

from water.utils import parse as p
from django.db.models import Count

In [64]:
filename_all = '../../../../data/datasets/all_parsed_cleaned.csv'
df_all_raw = pd.read_csv(filename_all).sort_values(['province', 'reservoir', 'ds'])
name_df = df_all_raw.reservoir.unique()


In [112]:
filename_all = '../../../../data/datasets/all_parsed_cleaned.csv'
df_all_raw = pd.read_csv(filename_all).sort_values(['province', 'reservoir', 'ds'])
name_df = df_all_raw.reservoir.unique()

file = '../../../../data/reservoirs/InfGeografica/InfVectorial/Gpkg/Inv_presas_embalses.gpkg'

# Open the file in geopandas
gdf_raw = gpd.read_file(file)
name_geo = gdf_raw[gdf_raw.nombre.notnull()].nombre.unique()

replace_vals = ['embalse del ', 'embalse de ', 'embalse ']

def clean_name(name):
    for val in replace_vals:
        name = name.lower().replace(val, '')
    return name

names_geo_dict = {name: clean_name(name) for name in name_geo}

In [73]:
name_df

array(['beninar', 'cuevas de almanzora', 'almodovar', 'arcos', 'barbate',
       'bornos', 'celemin', 'charco redondo', 'guadalcacin',
       'guadarranque', 'los hurones', 'zahara', 'zahara - el gastor',
       'arenoso', 'bembezar', 'guadalmellato', 'guadanunno', 'iznajar',
       'la brenna', 'martin gonzalo', 'puente nuevo', 'retortillo',
       'san rafael de navallana', 'sierra boyera', 'yeguas', 'beznar',
       'canales', 'colomera', 'cubillas', 'el portillo',
       'francisco abellan', 'los bermejales', 'negratin', 'quentar',
       'rules', 'san clemente', 'andevalo', 'aracena', 'chanza',
       'corumbel bajo', 'jarrama', 'los machos', 'piedras', 'zufre',
       'aguascebas', 'dannador', 'giribaile', 'guadalen', 'guadalmena',
       'jandula', 'la bolera', 'la fernandina', 'quiebrajano', 'rumblar',
       'tranco de beas', 'vadomojon', 'viboras', 'casasola',
       'conde de guadalhorce', 'el limonero', 'guadalhorce', 'guadalteba',
       'la concepcion', 'la vinnuela', 'ca

{'Embalse de Benínar': 'beninar',
 'Embalse de Cuevas de Almanzora': 'cuevas de almanzora',
 'Embalse de Almodóvar': 'almodovar',
 'Embalse de Arcos': 'arcos',
 'Embalse de Barbate': 'barbate',
 'Embalse de Bornos': 'bornos',
 'Embalse del Celemín': 'celemin',
 'Embalse de Charco Redondo': 'charco redondo',
 'Embalse de Guadalcacín': 'guadalcacin',
 'Embalse de Guadarranque': 'guadarranque',
 'Embalse de los Hurones': 'los hurones',
 'Embalse de Zahariche': 'zahara',
 'Embalse de Zahara - El Gastor': 'zahara - el gastor',
 'Embalse del Arenoso': 'arenoso',
 'Embalse del Bembézar': 'bembezar',
 'Embalse de Guadalmellato': 'guadalmellato',
 'Embalse de Guadalén': 'guadanunno',
 'Embalse de Iznájar': 'iznajar',
 'Embalse de la Feda': 'la brenna',
 'Embalse de Martín Gonzalo': 'martin gonzalo',
 'Embalse de Puente Nuevo': 'puente nuevo',
 'Embalse del Retortillo': 'retortillo',
 'Embalse de San Rafael de Navallana': 'san rafael de navallana',
 'Embalse de Sierra Boyera': 'sierra boyera',
 

cod_emb                                                     1.0
nombre                                Embalse de Tranco de Beas
cod_est                                                     E01
cod_roea                                                   5001
idemb_cbrh                                            4000115.0
cod_massup                                         ES0511100055
fuente_nom      IDE Confederación Hidrográfica del Guadalquivir
area                                                 17415295.0
perimetro                                               88713.0
fecha_alta                                                 2010
fecha_baja                                                    0
geometry      MULTIPOLYGON (((523504.7671098917 4229365.8424...
Name: 0, dtype: object

In [153]:
!ls -alh ../../../frontend/src/data 

total 320
drwxr-xr-x@  4 cg  staff   128B Mar 27 16:51 .
drwxr-xr-x@ 13 cg  staff   416B Mar 27 16:40 ..
-rw-r--r--@  1 cg  staff    81K Mar 27 16:50 reservoirs.geojson
-rw-r--r--@  1 cg  staff    74K Mar 27 16:51 reservoirs.json


In [162]:
# type: ignore
filename = '../../../frontend/src/data/reservoirs.json'
gdf = gdf_raw[gdf_raw.nombre.isin(df_matches['name_geo_full'])].copy()
gdf['name_data'] = [dict_matches[n] for n in gdf.nombre]
gdf.geometry = gdf.geometry.simplify(100)
# Turn this into a geo crs
gdf = gdf.to_crs("EPSG:4326")
# Reduce the storage size by making the geometry less granular

gdf.to_file(filename, driver='GeoJSON')
gdf.iloc[0]

cod_emb                                                     1.0
nombre                                Embalse de Tranco de Beas
cod_est                                                     E01
cod_roea                                                   5001
idemb_cbrh                                            4000115.0
cod_massup                                         ES0511100055
fuente_nom      IDE Confederación Hidrográfica del Guadalquivir
area                                                 17415295.0
perimetro                                               88713.0
fecha_alta                                                 2010
fecha_baja                                                    0
geometry      POLYGON ((-2.731509444260477 38.2119503448655,...
name_data                                        tranco de beas
Name: 0, dtype: object

In [163]:
gdf.name_data.unique()

array(['tranco de beas', 'conde de guadalhorce', 'la brenna',
       'la fernandina', 'guadalmena', 'giribaile', 'aguascebas',
       'san clemente', 'el portillo', 'la bolera', 'negratin', 'dannador',
       'guadanunno', 'quiebrajano', 'rumblar', 'vadomojon',
       'sierra boyera', 'jandula', 'yeguas', 'martin gonzalo',
       'guadalmellato', 'san rafael de navallana', 'viboras',
       'puente nuevo', 'bembezar', 'retortillo', 'canales', 'quentar',
       'colomera', 'cubillas', 'los bermejales', 'iznajar', 'gergal',
       'zahara', 'jose toran', 'puebla de cazalla', 'el pintado',
       'melonares', 'aracena', 'zufre', 'la minilla', 'cala', 'el agrio',
       'torre del aguila', 'la concepcion', 'chanza', 'andevalo',
       'guadalcacin', 'zahara - el gastor', 'bornos', 'arcos',
       'los hurones', 'barbate', 'guadarranque', 'celemin',
       'charco redondo', 'almodovar', 'guadalteba', 'beninar', 'casasola',
       'huesna', 'beznar', 'la vinnuela', 'el limonero', 'corumbel b

In [50]:
RainFall.objects.all()

<QuerySet []>

In [46]:
ReservoirState.objects.all()[0].reservoir.name

'conde de guadalhorce'

In [33]:
reservoirs = Reservoir.objects.annotate(
        num_states=Count("reservoir_reservoirstate")
    )

# Get all reservoirs with num_states > 0
reservoirs = reservoirs.filter(num_states__gt=0)
reservoirs[0].num_states


100

In [36]:
reservoirs[0].name

'conde de guadalhorce'

In [20]:
np.power(100000, 1/3)

46.41588833612778

In [ ]:
cm = 1000
hm3 = 100000 * cm

